<a href="https://colab.research.google.com/github/l-Monarch-l/Laborat/blob/main/GPT_FOR_LAB_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from collections import OrderedDict

In [3]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.layer_sizes = [input_size] + hidden_sizes + [output_size]
        self.weights = []
        self.biases = []

        for i in range(len(self.layer_sizes)-1):
            scale = np.sqrt(2.0 / (self.layer_sizes[i] + self.layer_sizes[i+1]))
            self.weights.append(np.random.randn(self.layer_sizes[i], self.layer_sizes[i+1]) * scale)
            self.biases.append(np.zeros(self.layer_sizes[i+1]))

    def relu(self, x):
        return np.maximum(0, x)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.activations = [X]
        self.z_values = []

        for i in range(len(self.weights)-1):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            a = self.relu(z)
            self.z_values.append(z)
            self.activations.append(a)

        z = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        a = self.softmax(z)
        self.z_values.append(z)
        self.activations.append(a)

        return a

    def backward(self, X, y, learning_rate, l2_lambda=0.001):
        m = X.shape[0]
        y_onehot = np.eye(self.layer_sizes[-1])[y]

        reg_term = l2_lambda * sum(np.sum(w**2) for w in self.weights) / (2*m)

        dZ = self.activations[-1] - y_onehot
        gradients = []

        dW = np.dot(self.activations[-2].T, dZ)/m + (l2_lambda * self.weights[-1])/m
        db = np.sum(dZ, axis=0)/m
        gradients.insert(0, (dW, db))

        for i in range(len(self.weights)-1, 0, -1):
            dA = np.dot(dZ, self.weights[i].T)
            dZ = dA * (self.activations[i] > 0).astype(float)
            dW = np.dot(self.activations[i-1].T, dZ)/m + (l2_lambda * self.weights[i-1])/m
            db = np.sum(dZ, axis=0)/m
            gradients.insert(0, (dW, db))

        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * gradients[i][0]
            self.biases[i] -= learning_rate * gradients[i][1]

        return reg_term

In [6]:
class SCPFoundationSystem:
    def __init__(self):
        self.params_options = OrderedDict([
            ('Тип объекта', ['гуманоид', 'растение', 'артефакт', 'сущность', 'животное', 'механизм']),
            ('Уровень опасности', ['низкий', 'средний', 'высокий', 'экстремальный']),
            ('Тип аномалии', ['биологическая', 'пространственная', 'временная', 'концептуальная', 'информационная']),
            ('Агрессивность', ['пассивная', 'умеренная', 'агрессивная', 'крайне агрессивная']),
            ('Подвижность', ['неподвижный', 'низкая', 'средняя', 'высокая']),
            ('Разумность', ['отсутствует', 'животный', 'человеческий', 'сверхразум']),
            ('Уровень аномалии', ['D-класс', 'C-класс', 'B-класс', 'A-класс', 'Ω-класс']),
            ('Регенерация', ['нет', 'медленная', 'средняя', 'быстрая', 'мгновенная']),
            ('Влияние на сознание', ['нет', 'слабое', 'умеренное', 'сильное', 'критическое'])
        ])

        self.classifier = NeuralNetwork(
            input_size=9,  # 9 параметров
            hidden_sizes=[16, 16],
            output_size=3  # Safe, Euclid, Keter
        )

        self.scp_database = OrderedDict()
        self.next_scp_id = 1
        self.train_classifier()

    def train_classifier(self):
        X = []
        y = []

        X.append([0, 0, 0, 0, 0, 1, 0, 0, 0])
        y.append(0)

        X.append([2, 0, 4, 0, 0, 0, 1, 0, 0])
        y.append(0)

        X.append([3, 1, 2, 1, 2, 2, 2, 1, 1])
        y.append(1)

        X.append([0, 3, 3, 3, 3, 3, 4, 4, 4])
        y.append(2)

        X_train = np.array(X)
        y_train = np.array(y)

        # Обучение с выводом прогресса
        for epoch in range(3000):
            output = self.classifier.forward(X_train)
            loss = -np.mean(np.log(output[np.arange(len(y_train)), y_train] + 1e-8))

            if epoch % 300 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

            self.classifier.backward(X_train, y_train, 0.005)

    def predict_scp_class(self, params):
        input_data = [self.params_options[key].index(value)
                     for key, value in params.items()]

        weights = [1.0, 1.5, 1.3, 1.4, 1.0, 1.2, 1.7, 1.6, 1.5]
        weighted_input = np.array(input_data) * weights

        prediction = self.classifier.forward(np.array([weighted_input]))
        return ['Safe', 'Euclid', 'Keter'][np.argmax(prediction)]

    def generate_protocol(self, params, scp_class):
        template = {
            'Safe': {
                'base': "Стандартный протокол содержания. {}",
                'special': "Дополнительные меры: {}"
            },
            'Euclid': {
                'base': "Протокол повышенной опасности. {}",
                'special': "Требуется охрана уровня {}"
            },
            'Keter': {
                'base': "ЭКСТРЕМАЛЬНЫЙ ПРОТОКОЛ. {}",
                'special': "КРИТИЧЕСКИЕ МЕРЫ: {}"
            }
        }

        details = []

        # Условия для особых мер
        if params['Регенерация'] in ['быстрая', 'мгновенная']:
            details.append("подавление регенерации")
        if params['Влияние на сознание'] in ['сильное', 'критическое']:
            details.append("персонал в защите HI-класса")
        if params['Уровень аномалии'] in ['A-класс', 'Ω-класс']:
            details.append("ежедневный осмотр O5")

        protocol = template[scp_class]['base']
        if details:
            protocol += " " + template[scp_class]['special'].format(" | ".join(details))

        return protocol

    def add_scp(self):
        print("\nДобавление нового SCP объекта:")
        params = {}

        for param_name, options in self.params_options.items():
            print(f"\n{param_name}:")
            for i, option in enumerate(options, 1):
                print(f"{i}. {option}")

            while True:
                try:
                    choice = int(input("Выберите номер: ")) - 1
                    if 0 <= choice < len(options):
                        params[param_name] = options[choice]
                        break
                    print("Некорректный ввод, попробуйте еще раз")
                except ValueError:
                    print("Пожалуйста, введите число")

        scp_class = self.predict_scp_class(params)
        scp_id = f"SCP-{self.next_scp_id}"
        self.next_scp_id += 1

        self.scp_database[scp_id] = {
            **params,
            'Класс объекта': scp_class,
            'Протокол содержания': self.generate_protocol(params, scp_class)
        }

        print(f"\nОбъект {scp_id} успешно добавлен!")
        self.show_scp_details(scp_id)

    def show_scp_details(self, scp_id):
        if scp_id not in self.scp_database:
            print("Объект не найден!")
            return

        print(f"\n=== Полные данные объекта {scp_id} ===")
        for key, value in self.scp_database[scp_id].items():
            print(f"{key}: {value}")
        print("="*50)

    def list_scps(self):
        print("\n=== Список всех SCP объектов ===")
        for scp_id, data in self.scp_database.items():
            print(f"{scp_id}: {data['Тип объекта']} | Класс: {data['Класс объекта']} | Опасность: {data['Уровень опасности']}")
        print(f"Всего объектов: {len(self.scp_database)}\n")

    def interactive_menu(self):
        while True:
            print("\n=== Меню системы SCP Foundation ===")
            print("1. Добавить новый SCP объект")
            print("2. Просмотреть все объекты")
            print("3. Найти объект по ID")
            print("4. Выход")

            choice = input("Выберите действие: ")

            if choice == '1':
                self.add_scp()
            elif choice == '2':
                self.list_scps()
            elif choice == '3':
                scp_id = input("Введите ID SCP (например, SCP-100): ")
                self.show_scp_details(scp_id)
            elif choice == '4':
                print("Завершение работы системы.")
                break
            else:
                print("Некорректный ввод, попробуйте еще раз.")

In [7]:
if __name__ == "__main__":
    system = SCPFoundationSystem()
    system.interactive_menu()

Epoch 0, Loss: 1.5660
Epoch 300, Loss: 0.2498
Epoch 600, Loss: 0.0942
Epoch 900, Loss: 0.0458
Epoch 1200, Loss: 0.0273
Epoch 1500, Loss: 0.0184
Epoch 1800, Loss: 0.0135
Epoch 2100, Loss: 0.0105
Epoch 2400, Loss: 0.0084
Epoch 2700, Loss: 0.0070

=== Меню системы SCP Foundation ===
1. Добавить новый SCP объект
2. Просмотреть все объекты
3. Найти объект по ID
4. Выход
Выберите действие: 1

Добавление нового SCP объекта:

Тип объекта:
1. гуманоид
2. растение
3. артефакт
4. сущность
5. животное
6. механизм
Выберите номер: 2

Уровень опасности:
1. низкий
2. средний
3. высокий
4. экстремальный
Выберите номер: 4

Тип аномалии:
1. биологическая
2. пространственная
3. временная
4. концептуальная
5. информационная
Выберите номер: 1

Агрессивность:
1. пассивная
2. умеренная
3. агрессивная
4. крайне агрессивная
Выберите номер: 4

Подвижность:
1. неподвижный
2. низкая
3. средняя
4. высокая
Выберите номер: 4

Разумность:
1. отсутствует
2. животный
3. человеческий
4. сверхразум
Выберите номер: 3

Уров